<a href="https://colab.research.google.com/github/huisung/president_go_kr_petitions_scraping/blob/master/get_petition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1초이상..(사이트에 대기시간이 1초씩 설정되어 있는듯..)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [1]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

from bs4 import BeautifulSoup
import re

def get_petition(number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title'))
            )
        bs = BeautifulSoup(wd.page_source, 'html.parser')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}


if __name__ == '__main__' or 'google.colab' in sys.modules:
    print(get_petition(300))
    print(get_petition(582349))

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (123 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [2]:
import pandas as pd
def get_p(num = 583432, cnt = 2):
    rtn = []
    for idx, i in enumerate(range(num-cnt, num)):
        tmp = get_petition(i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
    return (rtn_c, rtn)

%time c, rtn = get_p()
%time rtn.append(list(get_petition(582349).values()))
%time pd.DataFrame(rtn, columns=c)

CPU times: user 102 ms, sys: 2.08 ms, total: 104 ms
Wall time: 3.5 s
CPU times: user 67.4 ms, sys: 707 µs, total: 68.1 ms
Wall time: 1.22 s
CPU times: user 2.76 ms, sys: 958 µs, total: 3.72 ms
Wall time: 4.1 ms


,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,583430,청원종료,조국 전 장관 관련 ****** 상장폐지된다면 국민개미들 다 죽습니다.,639,안녕하십니까. 먼저 이렇게 이곳 청와대 청원장소까지 오게 되어 깊은 슬픔과 후회가 ...,,인권/성평등,2019-11-04,2019-12-04,naver
1,583431,청원종료,공영방송을 선거로 정할 수 있게 해주십시오.,527,KBS가 사용하고 있는 공영방송의 타이틀을 국민이 선거로 바꿀 수 있게 해주십시오....,,문화/예술/체육/언론,2019-11-04,2019-12-04,twitter
2,582349,답변완료,나경원 자한당 원내대표의 각종 의혹에 대한 특검 요청!,365040,나경원 자한당 원내대표의 각종 의혹ㆍ논란들이\n일파만파 번지고 있습니다.\n\n야권...,다음은 나경원 자유한국당 원내대표의 각종 의혹에 대해 특검을 요청하신 청원에 대해 ...,정치개혁,2019-08-28,2019-09-27,naver
